In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/ScoreGuardian')

import core.SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType

#Mostrar los duplicados
def print_res(sparkf_df):
  import pandas as pd
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  pd_spark_df = sparkf_df.toPandas()
 
  return pd_spark_df

In [0]:
# Importando base y configuraciones iniciales
base_seguimiento = spark.sql("select distinct * from catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hippyme_f")
base_seguimiento = base_seguimiento.withColumn("CODMES", col("CODMES").cast("integer"))
base_seguimiento = base_seguimiento.withColumn("DEF5", when(col("DISTANCIA_DEF24") <= 5, lit(1)).otherwise(lit(0)))
base_seguimiento = base_seguimiento.withColumn("DEF4", when(col("DISTANCIA_DEF24") <= 4, lit(1)).otherwise(lit(0))) 

base_seguimiento.persist(StorageLevel.MEMORY_AND_DISK).count()

# Verificando duplicandos en el segundo Join
dup = base_seguimiento.groupBy("CODCLAVEOPECTA", "CODMES").count().alias("count").filter(col("count")>1)
print_res(dup)

In [0]:
pd_rbm='PD_RBM'
pd_ant='PD_APP_HIP_TRDPYME_2Q24_REP_SUITE' 
pd_cal='PD_APP_HIP_TRDPYME_2Q24_REP_FAB'
pd_trc='XB_ADC_POND85_F'
tipo_banda = 'Jeffrey'#Jeffrey Vasiseck
codmes_default = 202307
columna_monto='MTOAPROBADO_SOLES'
titulo='Calibración Applicant App Tradicional'
condiciones= "CODMES >= 201901 AND DEF_120_BLOQ_REF_0 = 0"

In [0]:
# Objeto de Seguimiento
Obj_Seg = rmm.MonitorScore(
  nombre_modelo         = 'Personas Modelo Applicant Tradicional',
  codigo_modelo         = 'MOD-BCP-20658',
  tier                  = 'II',
  detalle_seguimiento   = 'Seguimiento Applicant Tradicional',
  mes_seguimiento       = '202504',
  base_monitoreo        = base_seguimiento, #dataframe de spark de tu base de seguimiento 
  pd1                   = pd_cal, #
  pd2                   = pd_rbm, #
  # pd3                   = pd_rbm, #
  monto_credito         = columna_monto, #
  query_univ            = "DEF_120_BLOQ_REF_0=0 and codmes >202201 AND PD_APP_HIP_TRDPYME_2Q24_REP_FAB IS NOT NULL", #
  bandas                = tipo_banda, 
  codmes_default        = codmes_default, #
  meses_ventana_target  = 24, #  ventana de target
  meses_atraso_target   = 4  #  condicion de target
)

# 1. Calibración

In [0]:
Obj_Seg_cal, rc_1, rm_1, fc_1, fm_1 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202101 and segmento_banca_f='1. Afluente'",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD12', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   =  "np.where(calxcuentas['CODMES'].isin([202307, 202308, 202310]),0.005, met1_xc)", #All_proy met1_xc met2_xc met3_xc
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det1_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(calxcuentas['CODMES'].isin([202307, 202308, 202310]),met1_xm + 0.006, met1_xm*1.25)", #All_proy met1_xm met2_xm met3_xm
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = 'det1_xm',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  #fact_to_proy_xc    = [fc_af, fc_con],
  #fact_to_proy_xm    = [fm_af, fm_con],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.28,
  eje_y_xm           = 0.28,
  dim_grafico        = (27,6.5),
  punt_mora          = 85, #Tamaño de puntos de mora temprana
  etiquetas          =True,
  pos_etiquetas_xm   = 10,
  pos_etiquetas_xc   = 10,
  tamaño_etiqueta    = 14,
  width_bar          = 1,

  # Exportar factores para vista driver
  exportar_factores  = True,
)

In [0]:
Obj_Seg_cal, rc_2, rm_2, fc_2, fm_2 = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202101 and segmento_banca_f<>'1. Afluente'",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD12', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = "np.where( (calxcuentas['CODMES'] == 202306) | (calxcuentas['CODMES'] == 202307) | (calxcuentas['CODMES'] == 202308) ,met1_xc, met1_xc)", #All_proy met1_xc met2_xc met3_xc
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = 'det1_xc',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "np.where(met1_xm == 0 , met1_xm + 0.006, met1_xm)", #All_proy met1_xm met2_xm met3_xm
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = "det1_xc",
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  #fact_to_proy_xc    = [fc_af, fc_con],
  #fact_to_proy_xm    = [fm_af, fm_con],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.14,
  eje_y_xm           = 0.14,
  dim_grafico        = (27,6.5),
  punt_mora          = 85,
  etiquetas          =True,
  pos_etiquetas_xm   = 10,
  pos_etiquetas_xc   = 10,
  tamaño_etiqueta    = 14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
Obj_Seg_cal, rc, rm, fc, fm = Obj_Seg.EvolCal(
  # Configuración de filtro de uso y pd de calibración
  query_uso          = "CODMES>=202101",
  pd_seg             = pd_cal,
  pd_comparacion     = True,
  proys_def_temp     = 6,
  proys_mor_temp     = 0,
  rd_aux             = 'RD21',
  rd_aux2            = 'RD12', # Corregido
  mora_temp          = 'MORA_30_3',

  # Configuración de proyecciones por cuentas
  proy_def_temp_xc   = '',
  suav_def_temp_xc   = 1,
  proy_mor_temp_xc   = '',
  suav_mor_temp_xc   = 1,
  prof_hist_xc       = 14,

  # Configuración de proyecciones por montos
  proy_def_temp_xm   = "",
  suav_def_temp_xm   = 1,
  proy_mor_temp_xm   = '',
  suav_mor_temp_xm   = 1,
  prof_hist_xm       = 14,
  fact_to_proy_xc    = [fc_1, fc_2],
  fact_to_proy_xm    = [fm_1, fm_2],

  # Detalles del gráfico de calibración
  titulo             = titulo,
  pos_leyenda        = (0.5, -0.25),
  eje_y_xc           = 0.12,
  eje_y_xm           = 0.12,
  dim_grafico        = (27,6.5),
  punt_mora          = 85, 
  etiquetas          =True,
  pos_etiquetas_xm   =10,
  pos_etiquetas_xc   =10,
  tamaño_etiqueta    =14,

  # Exportar factores para vista driver
  exportar_factores  = True
)

In [0]:
rc

In [0]:
rm

In [0]:
driver = 'segmento_banca_f'
Obj_SegPSI, mix_n, mix_m, psi_c, psi_m = Obj_Seg.MixPSI(
  # Selección del driver y filtro de uso
  driver             = driver,
  query_uso          = 'CODMES >= 202101',
  cast_int           = False,

  # Ventana de construcción o más antigua
  codmes_inicio1     = 202101,
  codmes_fin1        = 202204,

  # Periodo reciente
  codmes_inicio2     = 202301,
  codmes_fin2        = 202404,

  # Detalles del gráfico
  titulo             = driver,
  dim_grafico        = (25,6.5),
  pos_leyenda        = (0.5, -0.25),
  borde              = 0.08
)

# 2. Discriminación

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PD_APP_HIP_TRDPYME_2Q24', col('PD_APP_HIP_TRDPYME_2Q24').cast('float'))

var_pd = ['PD_APP_HIP_TRDPYME_2Q24_REP_FAB', 'PD_APP_HIP_TRDPYME_2Q24', pd_rbm]
caldri_veh_app = Obj_Seg.EvolDisVar(
  # Parametros de figura individual
  query_filtro      = '',
  codmes_ini        = 202209,
  codmes_fin        = 202312,
  amplitud          = 24,
  disc_target       = [24, 21,18],
  umbral_sup        = 0.5,
  umbral_inf        = 0.4,
  moviles           = True,
  dim_grafico       = (22.5, 6),
  pos_leyenda       = (0.5, -0.35),
  etiquetas         = True,
  gini_abs          = False,
 
  # Parametros de figura final
  ncolumnas         = 3,
  nfilas            = 1,
  variables         = var_pd,
  dim_grafico_total = (30, 6),
  ncol_leyenda      = 5,
  vspacio           = -0.25,
  hspacio           = 4,
  tamaño_etiqueta   = 14 
)

# 3. Seguimiento de variables del modelo


**Variable**                      | **Descripción**                                                                 | **Peso**  | **Sentido** |
|----------------------------------|----------------------------------------------------------------------------------|-----------|-------------|
| XB_ADC_POND85_F                  | Score base                                                                      | 62.19%    | +           |
| EXP_PCT_EVOL_SHIP_U6M_RT_U24_A3 | Evolución del promedio de deuda no hipotecaria entre 6 y 24 meses               | 11.18%    | +           |
| RCC_PCT_UTL6_UTL24_RT_U24_A2     | Evolución de la utilización promedio de líneas entre 6 y 24 meses               | 9.29%     | +           |
| SLD_PRM_PAS_MIN_24_24_RT_U24_I   | Ratio entre el pasivo mínimo y el pasivo promedio en los últ. 24 meses         | 5.28%     | +           |
| FLAG_PYME                        | Flag de Fondeo BCP Pyme                                                         | 5.07%     | -           |

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('DEU_NOHIP_6_24', col('DEU_NOHIP_6_24').cast('float'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('UTIL_PROM_6_24', col('UTIL_PROM_6_24').cast('float'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('PAS_MIN_PROM24', col('PAS_MIN_PROM24').cast('float'))

# Corrigiendo deuda no hipotecaria
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEU_NOHIP_6_24", when(col("DEU_NOHIP_6_24") <0, 0).otherwise(col("DEU_NOHIP_6_24")))

var_cal = ["XB_ADC_POND85_REP_SALESFORCE", "DEU_NOHIP_6_24", "UTIL_PROM_6_24", "PAS_MIN_PROM24"]
var_cal_Q = Obj_Seg.CutVar(variables=var_cal, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

# Corrigiendo significado de missing
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("DEU_NOHIP_6_24_Q", when(col("DEU_NOHIP_6_24_Q") == "98. Missing", "5. Sin deuda NOHIP").otherwise(col("DEU_NOHIP_6_24_Q")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("UTIL_PROM_6_24_Q", when(col("UTIL_PROM_6_24_Q") == "98. Missing", "5. Sin utilización").otherwise(col("UTIL_PROM_6_24_Q")))

In [0]:
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('RCC_PCT_UTL6_UTL24_RT_U24', col('RCC_PCT_UTL6_UTL24_RT_U24').cast('float'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('EXP_PCT_EVOL_SHIP_U6M_RT_U24', col('EXP_PCT_EVOL_SHIP_U6M_RT_U24').cast('float'))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn('SLD_PRM_PAS_MIN_24_24_RT_U24', col('SLD_PRM_PAS_MIN_24_24_RT_U24').cast('float'))

# Corrigiendo deuda no hipotecaria
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RCC_PCT_UTL6_UTL24_RT_U24", when(col("RCC_PCT_UTL6_UTL24_RT_U24") <0, 0).otherwise(col("RCC_PCT_UTL6_UTL24_RT_U24")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RCC_PCT_UTL6_UTL24_RT_U24", when(col("RCC_PCT_UTL6_UTL24_RT_U24") <0, 0).otherwise(col("RCC_PCT_UTL6_UTL24_RT_U24")))

var_cal1 = ["XB_ADC_POND85_REP_SALESFORCE", "EXP_PCT_EVOL_SHIP_U6M_RT_U24_1", "RCC_PCT_UTL6_UTL24_RT_U24_1", "SLD_PRM_PAS_MIN_24_24_RT_U24_1"]
var_cal1_Q = Obj_Seg.CutVar(variables=var_cal1, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")

# Corrigiendo significado de missing
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RCC_PCT_UTL6_UTL24_RT_U24_1_Q", when(col("RCC_PCT_UTL6_UTL24_RT_U24_1_Q") == "98. Missing", "5. Sin deuda NOHIP").otherwise(col("RCC_PCT_UTL6_UTL24_RT_U24_1_Q")))
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("EXP_PCT_EVOL_SHIP_U6M_RT_U24_1_Q", when(col("EXP_PCT_EVOL_SHIP_U6M_RT_U24_1_Q") == "98. Missing", "5. Sin utilización").otherwise(col("EXP_PCT_EVOL_SHIP_U6M_RT_U24_1_Q")))

In [0]:
caldri = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = "(CODMES BETWEEN 202105 AND 202306) AND CODMES NOT IN (202211, 202305)",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = True,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 5, 
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.3),
  punt_mora          = 70,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 2,
  nfilas             = 2,
  variables          = var_cal_Q,
  dim_grafico_total  = (17, 9),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 0.5,
  filtro_var         ="not like '%issing%'",
  iv                 = True 
)

In [0]:
caldri = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = "CODMES BETWEEN 202207 AND 202307",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 0, 
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.3),
  punt_mora          = 70,
  proys_def_temp     =6,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 2,
  nfilas             = 2,
  variables          = var_cal1_Q,
  dim_grafico_total  = (16.5, 9),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 0.5,
  filtro_var         ="not like '%issing%'",
  iv                 = True 
)

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='CODMES >= 202011',
    cast_int          =True,
    codmes_inicio1    =202202,
    codmes_fin1       =202212,
    codmes_inicio2    =202501,
    codmes_fin2       =202505,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 2,
    nfilas            = 2,
    variables         = var_cal_Q,
    dim_grafico_total = (17, 10),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08,
    metrica_secundaria='Media'
)

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso         ='CODMES >= 202011',
    cast_int          =True,
    codmes_inicio1    =202112,
    codmes_fin1       =202204,
    codmes_inicio2    =202501,
    codmes_fin2       =202505,
    dim_grafico       =(25, 6),
    pos_leyenda       =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista        = 'M',
    ncolumnas         = 2,
    nfilas            = 2,
    variables         = var_cal1_Q,
    dim_grafico_total = (17, 10),
    ncol_leyenda      =5,
    tamaño_leyenda    = 8.25,
    vspacio           =-0.45,
    hspacio           =0,
    borde             = 0.08,
    # metrica_secundaria='Media'
)

# 4. Drivers de negocio

In [0]:
from pyspark.sql.functions import date_format, concat, lit, lpad

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    'GRADO_ESTUDIO',
    when(col("MAX_GRADO")=='BACHILLER', '1. BACHILLER')
    .when(col("MAX_GRADO")=='TITULO', '3. TITULO')
    .when(col("MAX_GRADO")=='MAESTRIA', '2. MAESTRIA')
    .otherwise('99. Missing')
)

PD_SEG='PD_APP_HIP_TRDPYME_2Q24_REP'
PAUTA=335

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn(
    "SCR_XB",
    F.lit(174.24575241) + F.lit(57.707801636) * F.log((1 / col("PD_APP_HIP_TRDPYME_2Q24_REP")) - 1)
)


sol_credit = spark.table("catalog_lhcl_prod_bcp.bcp_udv_int_vu.m_solicitudcreditohipotecario")
Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.join(
    sol_credit.select(
        concat(lit('2'), lpad(sol_credit.codsolicitud, 10, '0')).alias('codsolicitud'),
        col("destipfinalidadcreditohipotecario")
    ),
    on="codsolicitud",
    how='left'
)

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("PAUTA", when(col('SCR_XB') >= PAUTA , 1)
                                            .when(col('SCR_XB') < PAUTA , 0)
                                            .when(col('destipfinalidadcreditohipotecario')=='MIGRACION DE OTRO BANCO', "97. CDD")
                                            .when(col('SCR_XB').isNull(), "99. Missing"))

Obj_Seg.base_monitoreo = Obj_Seg.base_monitoreo.withColumn("RANGO_PD_REP", when(col(PD_SEG).isNull(), '99. Missing')
                   .when(col(PD_SEG) <= 0.0045641, '01. [0 - 0.45%]')
                   .when(col(PD_SEG) <= 0.0151879, '02. <0.45 - 1.51%]')
                   .when(col(PD_SEG) <= 0.0353823, '03. <1.51 - 3.53%]')
                   .when(col(PD_SEG) <= 0.0581041, '04. <3.53 - 5.81%]')
                   .when(col(PD_SEG) > 0.0581041, '05. <5.81 - 100%]')
                   .when(col('destipfinalidadcreditohipotecario')=='MIGRACION DE OTRO BANCO', "97. CDD")
                   .otherwise('98. Otro'))

In [0]:
dri_neg = ["LTV_CORREGIDO_VF", "SUITE_AntLaboralMeses"]
dri_neg_Q = Obj_Seg.CutVar(variables=dri_neg, num_cortes=4, query_corte="CODMES BETWEEN 202101 AND 202410 AND DEF_120_BLOQ_REF_0 = 0")
dri_neg_Q = dri_neg_Q + ["MARCA_HML_T", "FLG_SOLTERO_ESPJ", "PLAZO_R", "SUITE_Edad_J_Q", "FLG_PDH_2_3", "PAUTA", "RANGO_PD_REP"]

In [0]:
caldri = Obj_Seg.CalVar(
  # Parametros de figura individual
  query_uso          = "(CODMES BETWEEN 202103 AND 202304) AND CODMES BETWEEN 202212 AND 202306",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = True,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 5, 
  cast_int           = True,
  dim_grafico        = (25, 6),
  pos_leyenda        = (0.5, -0.3),
  punt_mora          = 70,
 
  # Parametros de figura final
  tipo_vista         = 'M',
  ncolumnas          = 3,
  nfilas             = 3,
  variables          = dri_neg_Q,
  dim_grafico_total  = (25, 15),
  ncol_leyenda       = 5,
  etiquetas_total    = True, 
  pos_etiqueta       = 0.05,
  vspacio            = -0.5,
  hspacio            = 0.5,
  filtro_var         ="not like '%issing%'",
  iv                 = True 
)

In [0]:
caldri_veh_app, rc_dri, rm_dri = Obj_Seg.CalDri(
  driver             = 'MARCA_HML_T',
  query_uso          = f"CODMES BETWEEN 202206 AND 202306",
  pd_seg             = pd_cal,
  rd_aux             = 'RD12',
  pd_comparacion     = False,
  factores_cuentas   = fc,
  factores_montos    = fm,
  mat_importancia    = 5,
  cast_int           = True, 
  proys_def_temp     = 3,
  titulo             = f'MARCA_HML_T',
  dim_grafico        = (22, 6.5),
  etiquetas          = True,
  pos_etiquetas_xc   = [10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_etiquetas_xm   = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
  pos_leyenda        = (0.5, -0.2),
  punt_mora          = 100,
  iv = True
)

In [0]:
rc_dri

In [0]:
psi_dri = Obj_Seg.MixPSIVar(
  # Parametros de figura individual
    query_uso       ='CODMES >= 202201 AND CODMES <> 202407',
    cast_int        =True,
    codmes_inicio1  =202201,
    codmes_fin1     =202212,
    codmes_inicio2  =202301,
    codmes_fin2     =202307,
    dim_grafico     =(22.5, 6),
    pos_leyenda     =(0.5, -0.35),
 
    # Parametros de figura final
    tipo_vista      = 'M',
    ncolumnas       = 3,
    nfilas          = 3,
    variables       = dri_neg_Q,
    dim_grafico_total = (25, 15),
    ncol_leyenda    =8,
    tamaño_leyenda  = 8.25,
    vspacio         =-0.25,
    hspacio         =0,
    borde           = 0.08
)